In [65]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import math
import plotly.graph_objects as go
import plotly.express as px
#from functions import *
sns.set_style('whitegrid')

In [123]:
stocks = ["GOOG", "AAPL", "AMZN"]
start_date = '2018-01-01'
end_date = '2020-12-31'

In [77]:
## A FUNCTION TO RETRIEVE INFORMATION FOR A GIVEN LIST OF STOCKS

def fetch_data(stock_list, start_date, end_date):
    
    """Returns stock_price_df, stock_dividend_df"""
    
    if type(stock_list)!=list:
        stock_list = [stock_list]
    
    for i in range(len(stock_list)):
        stock_list[i] = stock_list[i].upper()

    stock_price = {}
    stock_dividend = {}
    for stock in stock_list:
        ticker = yf.Ticker(stock)
        stock_data = ticker.history(start=start_date, end=end_date)
        stock_price[stock] = stock_data['Close']
        stock_dividend[stock] = stock_data['Dividends']
        
    return pd.DataFrame(stock_price), pd.DataFrame(stock_dividend)

In [124]:
stock_prices, stock_dividend = fetch_data(stocks, start_date=start_date, end_date=end_date)

In [125]:
stock_prices

,GOOG,AAPL,AMZN
Date,,,
2018-01-02 00:00:00-05:00,53.250000,40.670971,59.450500
2018-01-03 00:00:00-05:00,54.124001,40.663887,60.209999
2018-01-04 00:00:00-05:00,54.320000,40.852779,60.479500
2018-01-05 00:00:00-05:00,55.111500,41.317898,61.457001
2018-01-08 00:00:00-05:00,55.347000,41.164436,62.343498
...,...,...,...
2020-12-23 00:00:00-05:00,86.619003,128.523254,159.263504
2020-12-24 00:00:00-05:00,86.942497,129.514465,158.634506
2020-12-28 00:00:00-05:00,88.804497,134.146622,164.197998


In [126]:
fig = go.Figure()

def plot_price(stock, fig):
    
    ## PLOTTING THE PRICES
    fig.add_trace(go.Scatter(x=stock_prices.index, y=stock_prices[stock], mode='lines', name=stock))
    
    ## PLOTTING THE DIVIDEND ROLLOUT
    dividend_rollout_days = stock_dividend[stock_dividend[stock] != 0].index
    price_when_dividend = stock_prices.loc[dividend_rollout_days, stock]
    fig.add_trace(go.Scatter(x=dividend_rollout_days, y=price_when_dividend, 
                             mode='markers', marker=dict(symbol='circle-open',size=10), 
                             showlegend=False, name="{} Dividend".format(stock)))

    fig.update_layout(
        title='Stock Price with Dividend Rollout',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        plot_bgcolor='white',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        hovermode="x unified"
        #hoverlabel=dict(bgcolor='white', font=dict(color='black'))
    )

for stock in stocks:
    plot_price(stock, fig)

fig.show()